# API

En este notebook se prepararán los datasets que servirán de insumo para las consultas a la API.

In [1]:
# Importar las librerias necesarias.
import pandas as pd
import html

## PlayTimeGenre

**Consigna:**

__def PlayTimeGenre(genero : str):__ Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Algunas consideraciones:
* El **género del juego** está en la columna "genres" del dataset "steam_games_procesado.csv".
* El **año de lanzamiento** está en la columna "year" del dataset "steam_games_procesado.csv".
* El **tiempo de juego** está en la columna "playtime_forever" del dataset "user_items_procesado.csv".

* Podemos vincular ambos datasets a través de la columna **"id"** de "steam_games_procesado.csv" y la columna **"item_id"** de "user_items_procesado.csv".

In [2]:
# Cargar steam_games_procesado.csv.
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")
games_df = games_df[["id", "year", "genres"]]

# Visualizar los primeros datos.
games_df.head()

,id,year,genres
0,761140,2018.0,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,2018.0,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,2017.0,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,2017.0,"['Action', 'Adventure', 'Casual']"
4,773570,NaN,NaN


In [3]:
# Obtener información de games_df.
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32125 entries, 0 to 32124
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      32125 non-null  int64  
 1   year    29955 non-null  float64
 2   genres  28850 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 753.1+ KB


In [4]:
# Analizar los valores nulos.
games_df.isna().sum()

id           0
year      2170
genres    3275
dtype: int64

Para esta consulta no nos sirven aquellos registros que tengan valores nulos en __year__ o __genres__, por lo que los eliminaremos.

In [5]:
# Borrar valores nulos.
games_df.dropna(subset=["year", "genres"], inplace=True, ignore_index=True)

# Castear year a entero.
games_df["year"] = games_df["year"].astype("int32")

Luego, para poder agrupar por género, necesitaremos que cada género ocupe una fila. Lograremos esto con la función __explode()__.

In [6]:
# Usar eval() para convertir las listas de strings entre comillas en listas de 
# cadenas de texto válidas antes de usar explode().
games_df['genres'] = games_df['genres'].apply(eval)

# explode() hace que cada género ocupe una fila distinta.
games_df = games_df.explode("genres")

# Resetear el índice
games_df.reset_index(inplace=True, drop=True)

# Visualizar los primeros resultados.
games_df.head()

,id,year,genres
0,761140,2018,Action
1,761140,2018,Casual
2,761140,2018,Indie
3,761140,2018,Simulation
4,761140,2018,Strategy


In [7]:
# Analizar los valores únicos en genres
games_df.genres.unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Massively Multiplayer', 'Early Access',
       'Animation &amp; Modeling', 'Video Production', 'Web Publishing',
       'Education', 'Software Training', 'Utilities',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [8]:
# Reemplazar algunos valores en la columna 'genres' después de descodificar HTML
# para mejorar la legibilidad.
games_df['genres'] = games_df['genres'].apply(lambda x: html.unescape(x))
games_df['genres'] = games_df['genres'].str.replace("'Animation & Modeling'", 'Animation & Modeling')
games_df['genres'] = games_df['genres'].str.replace("'Design & Illustration'", 'Design & Illustration')

In [9]:
# Obtener información de games_df.
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71207 entries, 0 to 71206
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      71207 non-null  int64 
 1   year    71207 non-null  int32 
 2   genres  71207 non-null  object
dtypes: int32(1), int64(1), object(1)
memory usage: 1.4+ MB


Ahora cargaremos el segundo dataframe.

In [10]:
# Cargar user_items_procesado.csv.
items_df = pd.read_csv("./Datasets/user_items_procesado.csv")

# Seleccionar columnas de interés para la API.
items_df = items_df[["item_id", "playtime_forever"]]

# Visualizar los primeros resultados.
items_df.head(3)

,item_id,playtime_forever
0,10,0.10
1,20,0.00
2,30,0.12


In [11]:
# Obtener información de items_df.
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094092 entries, 0 to 5094091
Data columns (total 2 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int64  
 1   playtime_forever  float64
dtypes: float64(1), int64(1)
memory usage: 77.7 MB


In [12]:
items_df.isna().sum() # No hay valores nulos

item_id             0
playtime_forever    0
dtype: int64

Vamos a sumar las horas jugadas para cada juego.

In [13]:
total_hours_by_game = items_df.groupby("item_id").playtime_forever.sum()

# Visualizar la serie total_hours_by_game.
total_hours_by_game

item_id
10        285131.65
20         16009.26
30         12606.34
40          2573.98
50         12109.32
            ...    
528580         0.00
528660         3.03
529670         0.00
529820         0.00
530720         0.00
Name: playtime_forever, Length: 10978, dtype: float64

Convertimos la serie anterior a un dataframe.

In [14]:
# Convertir la serie en DataFrame y resetear el índice.
items_df = total_hours_by_game.to_frame().reset_index()

# Visualizar los primeros resultados.
items_df.head(3)

,item_id,playtime_forever
0,10,285131.65
1,20,16009.26
2,30,12606.34


Procedemos a hacer un __inner join__ entre ambos DataFrames y asignaremos el resultado a PlayTimeGenre_df.

In [15]:
PlayTimeGenre_df = games_df.merge(items_df, left_on='id', right_on='item_id', 
                                  how='inner')

In [16]:
# Visualizar los primeros resultados de PlayTimeGenre_df.
PlayTimeGenre_df.head()

,id,year,genres,item_id,playtime_forever
0,282010,1997,Action,282010,155.32
1,282010,1997,Indie,282010,155.32
2,282010,1997,Racing,282010,155.32
3,70,1998,Action,70,44182.89
4,1640,2006,Strategy,1640,452.71


Podemos eliminar las columnas __id__ e __item_id__.

In [17]:
PlayTimeGenre_df.drop(["id", "item_id"], axis=1, inplace=True)

Ahora agruparemos por género y por año y sumaremos las horas de juego totales.

In [18]:
PlayTimeGenre_df = PlayTimeGenre_df.groupby(["genres", "year"]).playtime_forever.sum()

PlayTimeGenre_df

genres          year
Action          1983      57.92
                1984       6.40
                1988     266.68
                1989      10.11
                1990     305.63
                         ...   
Web Publishing  2013    5561.50
                2014     560.80
                2015    5811.27
                2016       2.27
                2017     156.37
Name: playtime_forever, Length: 343, dtype: float64

Usaremos la función __unstack(0)__ para pasar el primer índice de la serie multiindice a columnas.

In [19]:
PlayTimeGenre_df = PlayTimeGenre_df.unstack(0)

# Visualizar los primeros resultados de PlayTimeGenre_df.
PlayTimeGenre_df.head()

genres,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free to Play,Indie,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
year,,,,,,,,,,,,,,,,,,,,,
1983,57.92,57.92,NaN,NaN,57.92,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,6.40,6.40,NaN,NaN,6.40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987,NaN,128.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.92,NaN,NaN,NaN,NaN,NaN,NaN
1988,266.68,499.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.76,...,NaN,232.76,NaN,0.02,NaN,NaN,232.76,NaN,NaN,NaN
1989,10.11,155.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,84.74,NaN,NaN,NaN,NaN,NaN,NaN


Procederemos a guardar este DataFrame en un archivo .csv para ser consultado por la API.

In [20]:
PlayTimeGenre_df.to_csv("./Datasets_API/PlayTimeGenre.csv")

## UserForGenre

**Consigna:**

__def UserForGenre(genero : str):__ Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

Algunas consideraciones:
* El **tiempo de juego** está en la columna "playtime_forever" del dataset "user_items_procesado.csv".
* El **año** en el que está jugando está en la columna "review_year" del dataset "user_reviews_procesado.csv". Asumimos que el año en el que postea la review es el mismo año en el que está jugando.
* El **género del juego** está en la columna "genres" del dataset "steam_games_procesado.csv".

* Podemos vincular "user_reviews_procesado.csv" y "user_items_procesado.csv" a través de las columnas **"user_id"** e **"item_id"** que se encuentran en ambas tablas.
* Luego podemos vincular la columna **"item_id"** de esta tabla con la columna **"id"** de "steam_games_procesado.csv".

In [21]:
# Cargar user_items_procesado.csv.
items_df = pd.read_csv("./Datasets/user_items_procesado.csv")

# Visualizar los primeros resultados.
items_df.head()

,user_id,item_id,playtime_forever
0,76561197970982479,10,0.10
1,76561197970982479,20,0.00
2,76561197970982479,30,0.12
3,76561197970982479,40,0.00
4,76561197970982479,50,0.00


In [22]:
# Obtener información de items_df.
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094092 entries, 0 to 5094091
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           int64  
 2   playtime_forever  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 116.6+ MB


In [23]:
# Obtener filas y columnas de items_df.
items_df.shape

(5094092, 3)

In [24]:
# Cargar user_reviews_procesado.csv.
reviews_df = pd.read_csv("./Datasets/user_reviews_procesado.csv")

# Seleccionar columnas de interés.
reviews_df = reviews_df[["user_id", "item_id", "review_year"]]

# Visualizar los primeros resultados.
reviews_df.head()

,user_id,item_id,review_year
0,76561197970982479,1250,2011.0
1,76561197970982479,22200,2011.0
2,76561197970982479,43110,2011.0
3,js41637,251610,2014.0
4,js41637,227300,2013.0


In [25]:
# Obtener información de reviews_df.
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      58431 non-null  object 
 1   item_id      58431 non-null  int64  
 2   review_year  48498 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.3+ MB


Podemos ver valores nulos en __review_year__. Estos no nos servirán para la consulta y no podemos imputarlos porque no sabemos cuando hizo el review cada usuario asique optaremos por eliminarlos.

In [26]:
reviews_df.dropna(subset=["review_year"], inplace=True, ignore_index=True)

# Tras eliminar nulos podemos castear review_year a entero.
reviews_df.review_year = reviews_df.review_year.astype(int)

In [27]:
# Obtener información de reviews_df después de la transformación.
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48498 entries, 0 to 48497
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      48498 non-null  object
 1   item_id      48498 non-null  int64 
 2   review_year  48498 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


Ahora haremos un __inner join__ de estos 2 dataframes cuando haya coincidencias en las columnas __user_id__ e __item_id__.

In [28]:
it_rev_df = items_df.merge(reviews_df, on=["user_id", "item_id"], how="inner")

In [29]:
# Visualizar los primeros resultados de it_rev_df.
it_rev_df.head()

,user_id,item_id,playtime_forever,review_year
0,76561197970982479,22200,4.52,2011
1,76561197970982479,1250,166.77,2011
2,76561197970982479,43110,13.90,2011
3,js41637,227300,9.18,2013
4,js41637,239030,5.82,2013


In [30]:
# Obtener filas y columnas de it_rev_df.
it_rev_df.shape

(36293, 4)

Analicemos ahora el número de horas jugadas totales por usuario y por año.

In [31]:
it_rev_df.groupby(["user_id", "review_year"]).playtime_forever.sum()\
         .sort_values(ascending=False).head(6)

user_id             review_year
shinomegami         2015           11243.72
wolop               2011           10712.88
76561198039832932   2015           10563.43
tsunamitad          2015           10001.13
ThisIsWhereIGetOff  2014            8250.97
Nuclueus            2014            8216.85
Name: playtime_forever, dtype: float64

El número total de horas jugadas por año y por jugador no puede superar 8760 (horas en 1 año) o 8784 (horas en un año bisiesto). No obstante, vemos que los primeros 4 registros superan estos valores. Estos son datos agrupados, y es posible que solo algunos de los datos individuales sea incorrecto, pero como resultado, la suma de horas jugadas supera a las horas totales en 1 año.

Una de las posibles fuentes de error es la asunción que hicimos de que el año del review era el año en el que el usuario jugó. Como la respuesta de la API debe ser el número de horas jugadas por año, y ese número no puede ser superior a 8784, optaré por eliminar estos registros.

In [32]:
# Creamos un DataFrame con user_id, review_year y playtime_forever para las 4
# combinaciones que resultan en outliers y deben ser descartados.
user_year_outlier = it_rev_df.groupby(["user_id", "review_year"])\
                             .playtime_forever.sum()\
                             .sort_values(ascending=False).head(4)\
                             .to_frame().reset_index()

user_year_outlier

,user_id,review_year,playtime_forever
0,shinomegami,2015,11243.72
1,wolop,2011,10712.88
2,76561198039832932,2015,10563.43
3,tsunamitad,2015,10001.13


In [33]:
# Crear una lista vacía para almacenar los índices de las filas que cumplen con
# los criterios de outliers.
indices_outliers = []

# Iterar a través de las filas del DataFrame user_year_outlier.
for row in user_year_outlier.itertuples():
    user_id = row.user_id # Extraer el valor 'user_id' de la fila actual.
    review_year = row.review_year # Extrae el valor 'review_year' de la fila actual.

    # Iterar a través de las filas del DataFrame it_rev_df.
    for row2 in it_rev_df.itertuples():
        # Comprobar si los valores de user_id y review_year coinciden entre los DataFrames.
        if (row2.user_id == user_id) & (row2.review_year == review_year):
            # Si hay coincidencia, agrega el índice de la fila actual de 
            # 'it_rev_df' a la lista.
            indices_outliers.append(row2.Index)

# Visualizar indices_outliers.
indices_outliers

[13364, 13366, 13367, 4680, 10593, 10594, 10595, 10596, 10597, 10598, 12834]

Una vez identificados los índices de los outliers, pasamos a eliminar dichos registros de it_rev_df.

In [34]:
# Eliminar los registros cuyos índices estén en indices_outliers.
it_rev_df.drop(indices_outliers, inplace=True)

# Resetear el índice.
it_rev_df.reset_index(drop=True, inplace=True)

Pasamos ahora a cargar steam_games_procesado.csv.

In [35]:
# Cargar steam_games_procesado.csv.
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")

# Seleccionar columnas de interés.
games_df = games_df[["id", "genres"]]

# Cambiar el nombre de la columna id a item_id.
games_df.rename(columns={"id":"item_id"}, inplace=True)

# Visualizar los primeros resultados.
games_df.head(3)

,item_id,genres
0,761140,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."


In [36]:
# Analizar los valores nulos.
games_df.isna().sum()

item_id       0
genres     3275
dtype: int64

Para esta consulta no nos sirven aquellos registros que tengan valores nulos en __genres__, por lo que los eliminaremos.

In [37]:
# Borrar valores nulos.
games_df.dropna(subset=["genres"], inplace=True, ignore_index=True)

Luego, para poder agrupar por género, necesitaremos que cada género ocupe una fila. Lograremos esto con la función __explode()__.

In [38]:
# Usar eval() para convertir las listas de strings entre comillas en listas de 
# cadenas de texto válidas antes de usar explode().
games_df['genres'] = games_df['genres'].apply(eval)

games_df = games_df.explode("genres")

games_df.reset_index(inplace=True, drop=True) # Resetear el índice.

games_df.head() # Visualizar los primeros resultados.

,item_id,genres
0,761140,Action
1,761140,Casual
2,761140,Indie
3,761140,Simulation
4,761140,Strategy


In [39]:
# Mostrar los valores únicos en la columna genres.
games_df.genres.unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [40]:
# Reemplazar algunso valores en la columna 'genres' después de descodificar HTML
# para mejorar la legibilidad.
games_df['genres'] = games_df['genres'].apply(lambda x: html.unescape(x))
games_df['genres'] = games_df['genres'].str.replace("'Animation & Modeling'", 'Animation & Modeling')
games_df['genres'] = games_df['genres'].str.replace("'Design & Illustration'", 'Design & Illustration')

Vamos a hacer un __inner join__ de it_rev_df y games_df en su columna común: __item_id__.

In [41]:
it_rev_gam_df = it_rev_df.merge(games_df, on="item_id", how="inner")

# Visualizar los primeros resultados de it_rev_gam_df.
it_rev_gam_df.head()

,user_id,item_id,playtime_forever,review_year,genres
0,76561197970982479,22200,4.52,2011,Action
1,76561197970982479,22200,4.52,2011,Indie
2,seantheextraprawnsheepguy,22200,4.00,2011,Action
3,seantheextraprawnsheepguy,22200,4.00,2011,Indie
4,pipekissXD,22200,11.50,2013,Action


Vamos a usar it_rev_gam_df para responder la primera pregunta, _quién es el usuario que mas jugó por género_. Para ello debemos agrupar por género y por usuario y sumar las cantidad de horas jugadas.

In [42]:
it_rev_gam_df = it_rev_gam_df.groupby(["genres", "user_id"]).playtime_forever.sum()

it_rev_gam_df

genres          user_id            
Action          --000--                 49.15
                --ace--                 21.15
                --ionex--               14.03
                -2SV-vuLB-Kg           515.92
                -Beave-                 47.73
                                        ...  
Web Publishing  draoftwmate              0.95
                joshfeb06                5.07
                odoroitaka              20.73
                thetimegoesfastxd77     67.95
                tinbgr                   2.00
Name: playtime_forever, Length: 57133, dtype: float64

Usaremos la función __unstack(0)__ para pasar el primer índice de la serie multiindice a columnas.

In [43]:
it_rev_gam_df = it_rev_gam_df.unstack(0)

# Visualizar los primeros resultados.
it_rev_gam_df.head()

genres,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,Education,Free to Play,Indie,Massively Multiplayer,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
user_id,,,,,,,,,,,,,,,,,,,,
--000--,49.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,21.15,21.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.15,NaN,21.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,14.03,14.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.03,NaN,14.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,515.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.37,NaN,NaN,NaN
-Beave-,47.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Procederemos a guardar este dataframe en un .csv para ser consultado por la API.

In [44]:
it_rev_gam_df.to_csv("./Datasets_API/UserForGenre1.csv")

Vamos a usar it_rev_df para responder la segunda pregunta, _cuántas horas jugo por año el usuario obtenido en la primera consulta_. Para ello debemos agrupar por por usuario y por año y sumar las cantidad de horas jugadas.

In [45]:
it_rev_df = it_rev_df.groupby(["user_id", "review_year"]).playtime_forever.sum()

# Visualizar los primeros resultados.
it_rev_df.head()

user_id       review_year
--000--       2014            49.15
--ace--       2014            21.15
--ionex--     2015            14.03
-2SV-vuLB-Kg  2014           522.29
              2015             3.58
Name: playtime_forever, dtype: float64

Usaremos la función __unstack(0)__ para pasar el primer índice de la serie multiindice a columnas.

In [46]:
it_rev_df = it_rev_df.unstack(0)

# Visualizar los primeros resultados.
it_rev_df.head()

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Beave-,-GM-Dragon,-Kenny,-Mad-,-PRoSlayeR-,-SEVEN-,...,zuilde,zukuta,zumpo,zunbae,zuzuga2003,zv_odd,zvanik,zynxgameth,zyr0n1c,zzoptimuszz
review_year,,,,,,,,,,,,,,,,,,,,,
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,NaN,NaN,NaN,214.03,57.66,NaN,2618.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.1,311.28,NaN
2014,49.15,21.15,NaN,522.29,47.73,38.92,NaN,NaN,NaN,NaN,...,22.52,NaN,1382.68,1949.75,704.7,74.78,2.12,NaN,10.72,NaN


Procederemos a guardar este dataframe en un archivo .csv para ser consultado por la API.

In [47]:
it_rev_df.to_csv("./Datasets_API/UserForGenre2.csv")

## UsersRecommend

**Consigna:**

__def UsersRecommend(año : int):__ Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Algunas consideraciones:
* Tanto el **año** de la recomendación, el **número de recomendados** y el **análisis de sentimiento** están en "user_reviews_procesado.csv".
* El nombre del juego está en "steam_games_procesado.csv".
* Podemos vincular ambos datasets a través de la columna **"id"** de "steam_games_procesado.csv" y la columna **"item_id"** de "user_reviews_procesado.csv".
* Para determinar los juegos **más recomendados** usaremos como métrica el número de recomendaciones positivas (reviews.recommend == True) y no el análisis sentimiento, ya que el primero es completamente objetivo, mientras que el segundo depende de la libreria y el procesamiento y pueden obtenerse resultados diferentes.

In [48]:
# Cargar user_reviews_procesado.csv.
reviews_df = pd.read_csv("./Datasets/user_reviews_procesado.csv")

# Seleccionar columnas de interés-
reviews_df = reviews_df[["item_id", "recommend", "review_year"]]

# Visualizar los primeros resultados.
reviews_df.head()

,item_id,recommend,review_year
0,1250,True,2011.0
1,22200,True,2011.0
2,43110,True,2011.0
3,251610,True,2014.0
4,227300,True,2013.0


In [49]:
# Analizar nulos.
reviews_df.isna().sum()

item_id           0
recommend         0
review_year    9933
dtype: int64

Para esta consulta no nos sirven aquellos registros que tengan valores nulos en __year__ por lo que los eliminaremos.

In [50]:
# Borrar valores nulos.
reviews_df.dropna(inplace=True, ignore_index=True)

Ahora cargaremos steam_games_procesado.csv y lo uniremos con reviews_df para tener los nombres de los juegos.

In [51]:
# Cargar steam_games_procesado.csv.
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")

# Seleccionar columnas de interés.
games_df = games_df[["id", "app_name"]]

# Visualizar los primeros resultados.
games_df.head()

,id,app_name
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,773570,Log Challenge


Haremos un __inner join__ de games_df y reviews_df.

In [52]:
# Hacer un inner join de ambos dataframes.
UsersRecommend_df = games_df.merge(reviews_df, left_on='id', right_on='item_id', 
                                  how='inner')

# Eliminar la columna "item_id".
UsersRecommend_df.drop(columns=["item_id"], inplace=True)

# Visualizar los primeros resultados.
UsersRecommend_df.head(3)

,id,app_name,recommend,review_year
0,70,Half-Life,True,2015.0
1,70,Half-Life,True,2011.0
2,70,Half-Life,True,2014.0


Usaremos este mismo dataframe para la función UsersNotRecommend (ver próxima sección) por lo que haremos una copia ahora.

In [53]:
UsersNotRecommend_df = UsersRecommend_df.copy(deep=True)

Ahora agruparemos por año y por juego y sumaremos el número de recomendaciones.

In [54]:
UsersRecommend_df = UsersRecommend_df.groupby(["review_year", "app_name"]).recommend.sum()

UsersRecommend_df

review_year  app_name                 
2010.0       Alien Swarm                   4
             Amnesia: The Dark Descent     2
             ArcaniA                       1
             AudioSurf                     1
             Chime                         1
                                          ..
2015.0       sZone-Online                  2
             the static speaks my name    12
             theHunter Classic            13
             theHunter: Primal             8
             Астролорды: Облако Оорта      1
Name: recommend, Length: 4300, dtype: int64

Debemos volver a agrupar por año para calcular los 3 juegos con más recomendaciones positivas por año usando la función __nlargest()__.

In [55]:
# Agrupar por año y encontrar los 3 mayores valores.
UsersRecommend_df = UsersRecommend_df.groupby('review_year').nlargest(3)

# Eliminar el indice "review_year" que se duplica por la nueva agregación.
UsersRecommend_df = UsersRecommend_df.droplevel(0)

# Visualizar la serie multiindice obtenida.
UsersRecommend_df

review_year  app_name                        
2010.0       Team Fortress 2                       10
             Killing Floor                          6
             Alien Swarm                            4
2011.0       Team Fortress 2                       79
             Portal 2                              26
             Terraria                              25
2012.0       Team Fortress 2                      265
             Terraria                              42
             Garry's Mod                           38
2013.0       Team Fortress 2                      790
             Garry's Mod                          340
             Dota 2                               253
2014.0       Team Fortress 2                     1547
             Counter-Strike: Global Offensive    1068
             Garry's Mod                          757
2015.0       Counter-Strike: Global Offensive    1527
             Team Fortress 2                      634
             Garry's Mod            

Procederemos a guardar este dataframe en un .csv para ser consultado por la API.

In [56]:
UsersRecommend_df.to_csv("./Datasets_API/UsersRecommend.csv")

## UserNotRecommend

**Consigna:**

__def UsersNotRecommend(año : int):__ Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Algunas consideraciones:
* Trabajaremos con el dataframe UsersNotRecommend_df creado en la sección anterior.
* El mismo cuenta con las columnas "app_name", "recommend" y "review_year".
* Crearemos la columna "not_recommend" que será la negación de la columna "recommend". Es decir, cuando "recommend" sea False, "not_recommend" será True. Esto nos permitirá sumar el total de "no recomendaciones" y luego procederemos como en la sección anterior, agrupando por año y por juego y sumando las "no recomendaciones".

In [57]:
# Visualizar los primeros resultados.
UsersNotRecommend_df.head(3)

,id,app_name,recommend,review_year
0,70,Half-Life,True,2015.0
1,70,Half-Life,True,2011.0
2,70,Half-Life,True,2014.0


Creamos la columna __not_recommend__.

In [58]:
UsersNotRecommend_df["not_recommend"] = ~UsersNotRecommend_df["recommend"]

# Visualizamos 3 valores de UsersNotRecommend_df
UsersNotRecommend_df.sample(3)

,id,app_name,recommend,review_year,not_recommend
10193,570,Dota 2,True,2015.0,False
23021,261640,Borderlands: The Pre-Sequel,True,2014.0,False
39307,1250,Killing Floor,True,2014.0,False


Ahora eliminaremos la columna __recommend__.

In [59]:
UsersNotRecommend_df.drop(columns=["recommend"], inplace=True)

Ahora agruparemos por año y por juego y sumaremos el número de no recomendaciones.

In [60]:
# Crear una serie temporal agrupando por año de review y por nombre de la app
# y sumamos la columna not_recommend.
temp = UsersNotRecommend_df.groupby(["review_year", "app_name"])\
                                            .not_recommend.sum()

# Visualizar temp.
temp

review_year  app_name                 
2010.0       Alien Swarm                   0
             Amnesia: The Dark Descent     0
             ArcaniA                       0
             AudioSurf                     0
             Chime                         0
                                          ..
2015.0       sZone-Online                  1
             the static speaks my name     3
             theHunter Classic            13
             theHunter: Primal             3
             Астролорды: Облако Оорта      0
Name: not_recommend, Length: 4300, dtype: int64

Debemos volver a agrupar por año para calcular los 3 juegos con más recomendaciones negativas por año usando la función __nlargest()__.

In [61]:
# Agrupar por año y encontrar los 3 mayores valores.
temp = temp.groupby('review_year').nlargest(3)

# Eliminar el indice "review_year" que se duplica por la nueva agregación.
temp = temp.droplevel(0)

# Visualizar temp.ñ
temp

review_year  app_name                        
2010.0       Team Fortress 2                       1
             Alien Swarm                           0
             Amnesia: The Dark Descent             0
2011.0       And Yet It Moves                      2
             Team Fortress 2                       2
             Counter-Strike: Source                1
2012.0       Aliens vs. Predator™                  1
             Blacklight: Retribution               1
             Call of Duty®: Black Ops II           1
2013.0       Call of Duty®: Ghosts                14
             Team Fortress 2                      12
             Ace of Spades: Battle Builder        10
2014.0       DayZ                                 61
             Counter-Strike: Global Offensive     54
             Dota 2                               42
2015.0       PAYDAY 2                            150
             Counter-Strike: Global Offensive    129
             DayZ                                 72


Cuando hicimos el ETL, vimos que el número de no recomendados representaba el 11.5%. Esto se evidencia como un problema aquí al elegir los 3 juegos menos recomendados. Por ejemplo, mirando el año 2010, vemos que el primer lugar tiene solo 1 recomendación negativa, mientras que los puestos 2 y 3 no tienen ninguna, y ocupan ese lugar solo por orden alfabético.

Será necesario en este caso incorporar otra métrica, como el análisis de sentimiento. Para ello debemos cargar user_reviews_procesado.csv y unirlo a UsersNotRecommend_df.

In [62]:
del temp # Borrar el dataframe temp para optimizar recursos

In [63]:
# Cargar user_reviews_procesado.csv.
reviews_df = pd.read_csv("./Datasets/user_reviews_procesado.csv")

# Seleccionar columnas de interés.
reviews_df = reviews_df[["item_id", "sentiment_analysis"]]

# Visualizar los primeros resultados.
reviews_df.head(3)

,item_id,sentiment_analysis
0,1250,2
1,22200,2
2,43110,2


En sentiment_analysis, 0 representa negativo, 1 neutral y 2 positivo. 

Dado que __not_recommend__ representa una magnitud negativa, en el sentido de mientras mayor sea not_recommend, menos ha gustado el juego, transformaremos __sentiment_analysis__ para comportarse de manera similar. 

sentiment_analysis tomará el valor -1 cuando la reseña sea positiva, 0 cuando sea neutral y 1 cuando sea negativa.

De esta forma, mayores valores de __not_recommend__ y __sentiment_analysis__ representarán que el juego es menos gustado y recomendado y una combinación de ambas nos permitira obtener un score (__score_not_recommend__) que nos permitirá diferenciar entre juegos con el mismo número de not_recommend.

In [64]:
# Diccionario de reemplazos
reemplazos = {0: 1, 1: 0, 2: -1}

reviews_df['sentiment_analysis'] = reviews_df['sentiment_analysis'].replace(reemplazos)

In [65]:
# Visualizar los primeros resultados de reviews_df.
reviews_df.head(3)

,item_id,sentiment_analysis
0,1250,-1
1,22200,-1
2,43110,-1


Ahora agruparemos por __item_id__ para obtener el promedio de __sentiment_analysis__.

In [66]:
reviews_df = reviews_df.groupby("item_id").sentiment_analysis.mean()

# Visualizar los primeros resultados.
reviews_df.head()

item_id
10   -0.392857
20   -0.176471
30   -0.250000
40   -1.000000
50   -0.500000
Name: sentiment_analysis, dtype: float64

Antes de poder unir ambos dataframes, debemos agrupar UserNotRecommend_df por año, por nombre e id de juego y sumar la columna not_recommend.

In [67]:
UsersNotRecommend_df = UsersNotRecommend_df.groupby(["review_year", "app_name", "id"])\
                                            .not_recommend.sum()

UsersNotRecommend_df

review_year  app_name                   id    
2010.0       Alien Swarm                630        0
             Amnesia: The Dark Descent  57300      0
             ArcaniA                    39690      0
             AudioSurf                  12900      0
             Chime                      62100      0
                                                  ..
2015.0       sZone-Online               316390     1
             the static speaks my name  387860     3
             theHunter Classic          253710    13
             theHunter: Primal          322920     3
             Астролорды: Облако Оорта   385530     0
Name: not_recommend, Length: 4300, dtype: int64

In [68]:
# Resetear el indice
UsersNotRecommend_df = UsersNotRecommend_df.reset_index()

UsersNotRecommend_df

,review_year,app_name,id,not_recommend
0,2010.0,Alien Swarm,630,0
1,2010.0,Amnesia: The Dark Descent,57300,0
2,2010.0,ArcaniA,39690,0
3,2010.0,AudioSurf,12900,0
4,2010.0,Chime,62100,0
...,...,...,...,...
4295,2015.0,sZone-Online,316390,1
4296,2015.0,the static speaks my name,387860,3
4297,2015.0,theHunter Classic,253710,13
4298,2015.0,theHunter: Primal,322920,3


Ahora uniremos UsersNotRecommend_df y reviews_df.

In [69]:
UsersNotRecommend_df = UsersNotRecommend_df.merge(reviews_df, left_on="id", 
                                                  right_on="item_id",
                                                  how="inner")

# Visualizar los primeros resultados.
UsersNotRecommend_df.head()

,review_year,app_name,id,not_recommend,sentiment_analysis
0,2010.0,Alien Swarm,630,0,-0.5
1,2011.0,Alien Swarm,630,0,-0.5
2,2012.0,Alien Swarm,630,0,-0.5
3,2013.0,Alien Swarm,630,0,-0.5
4,2014.0,Alien Swarm,630,2,-0.5


En la tabla anterior tenemos para cada juego y año, el número de recomendaciones negativas y el promedio de análisis de sentimiento. Podemos crear ahora la columna __score_not_recommend__ que será la combinación lineal de las 2 últimas, otorgándoles el mismo peso.

Mientras mas alto sea este score, menos recomendados será el juego

In [70]:
UsersNotRecommend_df["score_not_recommend"] = UsersNotRecommend_df["not_recommend"]\
+ UsersNotRecommend_df["sentiment_analysis"] 

In [71]:
# Visualizar algunos resultados para UsersNotRecommend_df.
UsersNotRecommend_df.sample(3)

,review_year,app_name,id,not_recommend,sentiment_analysis,score_not_recommend
692,2013.0,Back to the Future: The Game,31290,0,0.0,0.0
2947,2015.0,STAR WARS™ Empire at War - Gold Pack,32470,0,-0.8,-0.8
1479,2013.0,Fortix,45400,0,0.0,0.0


A fin de tener mas claridad, eliminaremos las columnas __id__, __not_recommend__ y __sentiment_analysis__.

In [72]:
UsersNotRecommend_df.drop(columns=["id", "not_recommend", "sentiment_analysis"],
                          inplace=True)

# Visualizar los primeros resultados.
UsersNotRecommend_df.head()

,review_year,app_name,score_not_recommend
0,2010.0,Alien Swarm,-0.5
1,2011.0,Alien Swarm,-0.5
2,2012.0,Alien Swarm,-0.5
3,2013.0,Alien Swarm,-0.5
4,2014.0,Alien Swarm,1.5


Procederemos a guardar este DataFrame en un archivo .csv para ser consultado por la API.

In [73]:
UsersNotRecommend_df.to_csv("./Datasets_API/UsersNotRecommend.csv", index=False)

## sentiment_analysis

**Consigna:**

__def sentiment_analysis(año : int):__ Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

Algunas consideraciones:
* El **análisis de sentimientos** está en la columna "sentiment_analysis" del dataset "user_reviews_procesado.csv".
* El **año de lanzamiento** está en la columna "year" del dataset "steam_games_procesado.csv".
* Podemos vincular ambos datasets a través de la columna **"id"** de "steam_games_procesado.csv" y la columna **"item_id"** de "user_reviews_procesado.csv".

Cargamos el dataset de steam_games.

In [74]:
# Cargar steam_games_procesado.csv.
games_df = pd.read_csv("./Datasets/steam_games_procesado.csv")

# Seleccionar columnas de interés.
games_df = games_df[["id", "year"]]

# Visualizar los primeros resultados.
games_df.head()

,id,year
0,761140,2018.0
1,643980,2018.0
2,670290,2017.0
3,767400,2017.0
4,773570,NaN


In [75]:
# Analizar los valores nulos.
games_df.isna().sum()

id         0
year    2170
dtype: int64

Para esta consulta no nos sirven aquellos registros que tengan valores nulos en __year__ por lo que los eliminaremos.

In [76]:
# Borrar valores nulos.
games_df.dropna(subset=["year"], inplace=True, ignore_index=True)

# Castear "year" a entero.
games_df["year"] = games_df["year"].astype("int32")

In [77]:
# Obtener información de games_df.
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29955 entries, 0 to 29954
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      29955 non-null  int64
 1   year    29955 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 351.2 KB


Cargamos el dataset de user_reviews.

In [78]:
# Cargar user_reviews_procesado.csv.
reviews_df = pd.read_csv("./Datasets/user_reviews_procesado.csv")

# Seleccionar columnas de interés.
reviews_df = reviews_df[["item_id", "sentiment_analysis"]]

# Visualizar los primeros resultados.
reviews_df.head()

,item_id,sentiment_analysis
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,0


In [79]:
# Analizar los valores nulos.
reviews_df.isna().sum()

item_id               0
sentiment_analysis    0
dtype: int64

In [80]:
# Obtener información de reviews_df.
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   item_id             58431 non-null  int64
 1   sentiment_analysis  58431 non-null  int64
dtypes: int64(2)
memory usage: 913.1 KB


Para saber cuántos sentimientos negativos (0), neutrales (1) y positivos (2) hay por cada videojuego, tendremos que hacer __one-hot encoding__ en la columna __sentiment_analysis__.

In [81]:
# Crear variables dummies para sentiment_analysis.
sentiment_dummies = pd.get_dummies(reviews_df.sentiment_analysis).astype(int)

# Concatenar reviews_df con sentiment_dummies.
reviews_df = pd.concat([reviews_df, sentiment_dummies], axis = 1)

# Eliminar la columna "sentiment_analysis".
reviews_df.drop("sentiment_analysis", axis=1, inplace=True)

# Visualizar los primeros resultados.
reviews_df.head()

,item_id,0,1,2
0,1250,0,0,1
1,22200,0,0,1
2,43110,0,0,1
3,251610,0,0,1
4,227300,1,0,0


Vamos a agrupar por __item_id__ para saber el número de reseñas para cada juego.

In [82]:
reviews_df = reviews_df.groupby("item_id").sum()

# Visualizar los primeros resultados.
reviews_df.head(3)

,0,1,2
item_id,,,
10,4,26,26
20,4,6,7
30,1,1,2


In [83]:
# Resetear el indice para que "item_id" pase a ser una columna.
reviews_df.reset_index(inplace=True)

# Visualizar los primeros resultados.
reviews_df.head(3)

,item_id,0,1,2
0,10,4,26,26
1,20,4,6,7
2,30,1,1,2


Procedemos a hacer un __inner join__ de ambos dataframes.

In [84]:
sentiment_analysis_df = games_df.merge(reviews_df, left_on='id', right_on='item_id', 
                                  how='inner')

# Visualizar los primeros resultados.
sentiment_analysis_df.head(3)

,id,year,item_id,0,1,2
0,282010,1997,282010,0,1,0
1,70,1998,70,15,12,34
2,2400,2006,2400,6,5,12


Podemos eliminar las columnas __id__ e __item_id__.

In [85]:
sentiment_analysis_df.drop(["id", "item_id"], axis=1, inplace=True)

Ahora agruparemos por año y sumaremos el análisis de sentimientos de cada tipo.

In [86]:
sentiment_analysis_df = sentiment_analysis_df.groupby("year").sum()

# Visualizar los primeros resultados.
sentiment_analysis_df.head(3)

,0,1,2
year,,,
1989,0,0,1
1990,1,0,4
1991,0,0,1


Procederemos a guardar este dataframe en un archivo .csv para ser consultado por la API.

In [87]:
sentiment_analysis_df.to_csv("./Datasets_API/sentiment_analysis.csv")